In [2]:
# matpltlib 库以及参数
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams.update({'axes.labelsize': 'large'})
matplotlib.rcParams.update({'xtick.labelsize': 'large'})
matplotlib.rcParams.update({'ytick.labelsize': 'large'})
matplotlib.rcParams.update({'legend.fontsize': 'large'})
matplotlib.rcParams.update({'errorbar.capsize': 3})

#其他的库
import numpy as np
import pandas as pd
import scipy as sp 
from scipy.optimize import leastsq
#封装进度条的函数
from tqdm import tqdm  

COLORLIST1=['#0404B4','#0000ff','#6E6E6E','#ff0000','#B40404']
COLORLIST2=['#0000ff','#0404B4','#000000','#B40404','#ff0000']
COLORLIST3=['#0052ad','#0074f5','#68aefc','#828282','#000000']
COLORLIST4=['#ff0000','#ff7474','#a9a9a9','#848484','#000000']
MARKERLIST=['o','v','x','^','s']


In [19]:
df=pd.read_csv('./DATA/redshift_bin_org.csv',index_col=[0,1])
df_list=[]
for i in range(52):
    ##切开，赋值
    df_list.append(df.loc[i])
    #重排序
    df_list[i].sort_values(by=['g_SDSS-HSC'],ascending=True,inplace=True)
    #重命名索引
    df_list[i].reset_index(drop=True,inplace=True)


C:\Users\ZHms\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [29]:
n=5
df_list_first=[]
df_list_last=[]
for i in range(52):
    df_list_first.append(df_list[i].iloc[0:n])
    df_list_last.append(df_list[i].iloc[100-n:100])


In [32]:
df_list_last[1]


,SDSS_NAME,RA,DEC,REDSHIFT,g_SDSS,r_SDSS,i_SDSS,LOGBH,LOGEDD_RATIO,LOGLBOL,...,r_HSC,i_HSC,hsc_id,g_SDSS-HSC,r_SDSS-HSC,i_SDSS-HSC,MBH,EDD_RATIO,LBOL,half_g_SDSS+HSC
95,123742.97-003330.8,189.429078,-0.558558,0.4160,19.121,18.907,18.704,8.799878,-1.454947,45.445301,...,18.510487,18.433952,41140035734497536,0.414163,0.396513,0.270048,6.307795e+08,0.035079,2.788056e+45,18.913918
96,101609.65+014018.4,154.040213,1.671803,0.4268,19.457,19.528,19.066,8.526165,-1.327952,45.298584,...,18.781364,18.904514,43172328359612820,0.461614,0.746636,0.161486,3.358656e+08,0.046995,1.988766e+45,19.226193
97,021634.01-005759.2,34.141749,-0.966451,0.4370,19.753,19.636,19.430,7.917463,-0.866759,45.151075,...,19.323076,19.145643,40678094821941009,0.572651,0.312924,0.284357,8.269190e+07,0.135907,1.416037e+45,19.466675
98,122515.31-022237.2,186.313804,-2.377002,0.4357,19.212,19.198,18.945,8.477136,-1.194291,45.383216,...,18.743515,18.751904,40062093432470882,0.576582,0.454485,0.193096,3.000105e+08,0.063931,2.416665e+45,18.923709
99,020929.83-005602.6,32.374331,-0.934066,0.4270,20.326,19.940,19.625,8.987460,-2.293646,44.794185,...,19.611492,19.223629,40673834214380319,0.589413,0.328508,0.401371,9.715384e+08,0.005086,6.225653e+44,20.031294


In [37]:
first=pd.DataFrame(columns=df_list_first[0].columns)
for i in range(52):
    first=first.append(df_list_first[i])
    first.reset_index(drop=True,inplace=True)

In [38]:
first

,SDSS_NAME,RA,DEC,REDSHIFT,g_SDSS,r_SDSS,i_SDSS,LOGBH,LOGEDD_RATIO,LOGLBOL,...,r_HSC,i_HSC,hsc_id,g_SDSS-HSC,r_SDSS-HSC,i_SDSS-HSC,MBH,EDD_RATIO,LBOL,half_g_SDSS+HSC
0,122641.42-002005.0,186.672622,-0.334741,0.3534,19.047,18.936,18.970,8.236788,-1.090036,45.247122,...,20.194746,20.138531,41131106497489791,-1.742309,-1.258746,-1.168531,1.724995e+08,0.081276,1.766536e+45,19.918154
1,090550.30+003948.1,136.459595,0.663373,0.3538,20.090,19.400,18.931,9.392642,-2.238037,45.254976,...,20.278971,19.815674,42050152189419064,-1.629454,-0.878971,-0.884674,2.469688e+09,0.005780,1.798772e+45,20.904727
2,091357.26+052230.7,138.488584,5.375209,0.3461,19.342,19.190,19.025,7.941106,-0.861172,45.180305,...,19.977554,19.950109,45251934344462134,-1.382339,-0.787554,-0.925110,8.731855e+07,0.137666,1.514625e+45,20.033169
3,113416.81-001902.3,173.570044,-0.317331,0.3558,19.479,19.001,18.799,8.634890,-1.503374,45.231886,...,19.461918,19.446054,41091386639939128,-1.289028,-0.460918,-0.647054,4.314099e+08,0.031378,1.705636e+45,20.123514
4,143101.65-004251.6,217.756886,-0.714354,0.3339,18.967,18.840,18.824,8.787839,-1.636255,45.251954,...,19.795151,19.845528,41223456884294683,-1.229713,-0.955151,-1.021528,6.135348e+08,0.023107,1.786299e+45,19.581856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,141332.35-004909.5,213.384828,-0.819317,4.1381,21.112,19.588,19.292,9.519335,-0.328572,47.291134,...,20.010592,19.640831,41210262744752948,-0.744773,-0.422591,-0.348831,3.306247e+09,0.469276,1.954942e+47,21.484387
256,135422.99-003906.0,208.595831,-0.651683,4.4197,22.898,20.658,20.181,8.920940,0.111765,47.133076,...,21.012552,20.527344,41197343483134889,-0.663497,-0.354552,-0.346344,8.335651e+08,1.293497,1.358550e+47,23.229748
257,220842.95-010403.0,332.178972,-1.067518,3.9931,21.525,19.958,19.720,9.209170,-0.300498,47.009043,...,20.496029,20.241352,41561822997801489,-0.542171,-0.538029,-0.521352,1.618714e+09,0.500613,1.021040e+47,21.796086
258,224922.94-010745.8,342.345610,-1.129405,3.9899,21.051,19.764,19.507,9.443587,-0.445937,47.098020,...,20.372501,20.151985,41592746762328332,-0.503337,-0.608501,-0.644985,2.777069e+09,0.358148,1.253198e+47,21.302668
